In [1]:
from groq import Groq
import os
import json

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [3]:
client = Groq(
    api_key=os.environ['GROQ_API_KEY']
,
)
MODEL = 'llama3-70b-8192'

In [4]:
def get_response(question):
    return json.dumps({"question": question})


def mutual_fund(fund_name):
    """Get the details of a mutual fund"""
    if "growth" in fund_name.lower():
        return json.dumps({"fund_name": "Growth Fund", "nav": 150.75, "1_year_return": "12.5%"})
    elif "income" in fund_name.lower():
        return json.dumps({"fund_name": "Income Fund", "nav": 102.50, "1_year_return": "8.4%"})
    else:
        return json.dumps({"fund_name": fund_name, "details": "unknown"})
    

def upi(transaction_id):
    """Get the status of a UPI transaction"""
    if transaction_id == "TX123":
        return json.dumps({"transaction_id": "TX123", "status": "Success", "amount": 1500})
    else:
        return json.dumps({"transaction_id": transaction_id, "status": "Pending"})
    

def health_insurance(policy_number):
    """Get the details of a health insurance policy"""
    if policy_number == "HP001":
        return json.dumps({"policy_number": "HP001", "coverage": "500000", "premium": "12000", "status": "Active"})
    else:
        return json.dumps({"policy_number": policy_number, "details": "unknown"})
    

def cash_loan(loan_id):
    """Get the details of a cash loan"""
    if loan_id == "LN123":
        return json.dumps({"loan_id": "LN123", "amount": "20000", "interest_rate": "12%", "status": "Approved"})
    else:
        return json.dumps({"loan_id": loan_id, "details": "unknown"})

In [5]:
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_response",
                "description": "Responding a casual chat",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "Responding a casual chat",
                        }
                    },
                    "required": ["question"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "mutual_fund",
                "description": "Get the details of a mutual fund",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "fund_name": {
                            "type": "string",
                            "description": "The name of the mutual fund (e.g. 'Growth Fund')",
                        }
                    },
                    "required": ["fund_name"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "upi",
                "description": "Get the status of a UPI transaction",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "transaction_id": {
                            "type": "string",
                            "description": "The ID of the UPI transaction (e.g. 'TX123')",
                        }
                    },
                    "required": ["transaction_id"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "health_insurance",
                "description": "Get the details of a health insurance policy",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "policy_number": {
                            "type": "string",
                            "description": "The policy number of the health insurance (e.g. 'HP001')",
                        }
                    },
                    "required": ["policy_number"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "cash_loan",
                "description": "Get the details of a cash loan",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "loan_id": {
                            "type": "string",
                            "description": "The ID of the cash loan (e.g. 'LN123')",
                        }
                    },
                    "required": ["loan_id"],
                },
            },
        }
    ]

In [8]:
def run_conversation(user_prompt):
    """
    Executes a conversation with a language model, handling function calls to obtain
    additional data if required.

    Args:
        user_prompt (str): The initial input from the user.

    Returns:
        str: The final response generated by the language model, possibly including
        information retrieved from calling specific functions related to mutual funds,
        UPI transactions, health insurance policies, and cash loans.

    The function performs the following steps:
    1. Constructs an initial message list with system instructions and the user prompt.
    2. Sends these messages to the language model to generate a response.
    3. Checks if the response includes any tool calls (i.e., requests to call specific functions).
    4. If tool calls are present:
        a. Identifies and calls the appropriate function with the provided arguments.
        b. Appends the function response to the message list.
        c. Sends the updated messages back to the language model to generate a final response.
    5. If no tool calls are present, uses the initial response as the final response.
    6. Returns the final response generated by the language model.

    Note:
    - The available functions for tool calls are `get_response`, `mutual_fund`, `upi`,
      `health_insurance`, and `cash_loan`.
    """
    
    messages = [
        {
            "role": "system",
            "content": "You are a function calling LLM that uses the data extracted from the functions to answer questions around mutual funds, UPI transactions, health insurance policies, and cash loans. Don't mention anything about tool in response."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]

    
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        available_functions = {
            "get_response": get_response,
            "mutual_fund": mutual_fund,
            "upi": upi,
            "health_insurance": health_insurance,
            "cash_loan": cash_loan,
        }
        messages.append(response_message)

        for tool_call in tool_calls:
            # extracting the tool information
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                **function_args
            )
            
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )

        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        final_response = second_response.choices[0].message.content
        
    
    else:
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            max_tokens=4096
        )
        messages.append(response_message)
        final_response = response.choices[0].message.content

    return final_response

**Question about Mutual fund**

In [9]:
user_prompt = "Tell me about the Growth Fund mutual fund."
print(run_conversation(user_prompt))

The Growth Fund is an open-ended mutual fund scheme that aims to provide long-term capital appreciation by investing in a diversified portfolio of equities. 

As of the latest available data, the Net Asset Value (NAV) of the Growth Fund is ₹150.75. In terms of performance, the fund has delivered a return of 12.5% over the past 1 year. This suggests that the fund has been generating decent returns for its investors over the short to medium term.

Please let me know if you would like to know more about the fund's investment strategy, portfolio composition, or any other aspect. I'm here to help!


**Question about health insurance**

In [10]:
user_prompt = "Give me the details of health insurance policy HP001."
print(run_conversation(user_prompt))

Based on the extracted data, here are the details of health insurance policy HP001:

Policy Number: HP001
Coverage: ₹5,00,000
Premium: ₹12,000
Status: Active


**Question about UPI**

In [11]:
user_prompt = "What is the status of UPI transaction TX123?"
print(run_conversation(user_prompt))

The status of UPI transaction TX123 is "Success" and the amount is ₹1500.


**Question about cash loan**

In [12]:
user_prompt = "Provide details of the cash loan with ID LN123."
print(run_conversation(user_prompt))

Based on the information retrieved, here are the details of the cash loan with ID LN123:

* Loan ID: LN123
* Amount: ₹20,000
* Interest Rate: 12%
* Status: Approved


**Chit chat question**

In [13]:
user_prompt = "how are you"
print(run_conversation(user_prompt))

I'm doing well, thank you for asking! I'm ready to help answer any questions you may have about mutual funds, UPI transactions, health insurance policies, or cash loans. How can I assist you today?
